<a href="https://colab.research.google.com/github/liadekel/analyzing-big-data/blob/master/HW_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework Assignment 1
### [The Art of Analyzing Big Data - The Data Scientist’s Toolbox](https://www.ise.bgu.ac.il/labs/fire/lectures.html)
#### By Dr. Michael Fire 

For this homework you will need to write code that analyzes real-world datasets. The code needs to be written in Python using the [sqlite3](https://docs.python.org/2/library/sqlite3.html) package. 

**Please note:** You need to answer only the questions that match your ID first digit.

# 1. Babies Names Dataset (35pt)

**Task 1 (for everyone):** Write a code that uses the  [**babies names dataset**](https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-data-by-state-and-district-of-#topic=developers_navigation) and creates a table named (Names) with the dataset data and the following columns: 'State', 'Gender', 'Name', 'Number', and 'Year' (5pt)
**Bonus:** Load the data using a Batch INSERT SQL Query (2pt)

In [0]:
# restart state
! rm -rf ./datasets

In [56]:
# Creating a dataset directory
!mkdir ./datasets
!mkdir ./datasets/us-baby-name
# download the dataset using wget
!wget --directory-prefix ./datasets/us-baby-name https://www.ssa.gov/oact/babynames/state/namesbystate.zip
!unzip ./datasets/us-baby-name/*.zip  -d ./datasets/us-baby-name/namesbystate
# concatenate to one file
!cat ./datasets/us-baby-name/namesbystate/*.TXT >> ./datasets/us-baby-name/namesbystate.txt

--2020-03-24 08:52:54--  https://www.ssa.gov/oact/babynames/state/namesbystate.zip
Resolving www.ssa.gov (www.ssa.gov)... 137.200.4.16, 2001:1930:e03::aaaa
Connecting to www.ssa.gov (www.ssa.gov)|137.200.4.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21974087 (21M) [application/zip]
Saving to: ‘./datasets/us-baby-name/namesbystate.zip’

namesbystate.zip    100%[===================>]  20.96M  2.74MB/s    in 7.7s    

2020-03-24 08:53:02 (2.71 MB/s) - ‘./datasets/us-baby-name/namesbystate.zip’ saved [21974087/21974087]

Archive:  ./datasets/us-baby-name/namesbystate.zip
  inflating: ./datasets/us-baby-name/namesbystate/AK.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/AL.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/AR.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/AZ.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/CA.TXT  
  inflating: ./datasets/us-baby-name/namesbystate/CO.TXT  
  inflating: ./datasets/us-baby-name/na

In [0]:
import sqlite3
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline
TEXT_PATH = './datasets/us-baby-name/namesbystate.txt'
DB_PATH = './datasets/us-baby-name/namebystate.sqlite'

In [58]:
conn = sqlite3.connect(DB_PATH) # connecting to the database
c = conn.cursor() # creating a cursor object
# create Names table
c.execute(
        '''CREATE TABLE IF NOT EXISTS Names
             ([State] text,
              [Gender] text,
              [Year] integer,
              [Name] text,
              [Number] integer)
        '''
)

#load data into convinient format
with open(TEXT_PATH) as f:
    names = f.readlines()
names = [tuple(name.strip().split(",")) for name in names]

# insert data to db
c.executemany(
    '''INSERT INTO Names(State, Gender, Year, Name, Number)
       values (?,?,?,?,?)
    ''', names
)
# show all rows in Names
c.execute("SELECT * FROM Names").fetchall()

[('AK', 'F', 1910, 'Mary', 14),
 ('AK', 'F', 1910, 'Annie', 12),
 ('AK', 'F', 1910, 'Anna', 10),
 ('AK', 'F', 1910, 'Margaret', 8),
 ('AK', 'F', 1910, 'Helen', 7),
 ('AK', 'F', 1910, 'Elsie', 6),
 ('AK', 'F', 1910, 'Lucy', 6),
 ('AK', 'F', 1910, 'Dorothy', 5),
 ('AK', 'F', 1911, 'Mary', 12),
 ('AK', 'F', 1911, 'Margaret', 7),
 ('AK', 'F', 1911, 'Ruth', 7),
 ('AK', 'F', 1911, 'Annie', 6),
 ('AK', 'F', 1911, 'Elizabeth', 6),
 ('AK', 'F', 1911, 'Helen', 6),
 ('AK', 'F', 1912, 'Mary', 9),
 ('AK', 'F', 1912, 'Elsie', 8),
 ('AK', 'F', 1912, 'Agnes', 7),
 ('AK', 'F', 1912, 'Anna', 7),
 ('AK', 'F', 1912, 'Helen', 7),
 ('AK', 'F', 1912, 'Louise', 7),
 ('AK', 'F', 1912, 'Jean', 6),
 ('AK', 'F', 1912, 'Ruth', 6),
 ('AK', 'F', 1912, 'Alice', 5),
 ('AK', 'F', 1912, 'Esther', 5),
 ('AK', 'F', 1912, 'Ethel', 5),
 ('AK', 'F', 1912, 'Margaret', 5),
 ('AK', 'F', 1912, 'Marie', 5),
 ('AK', 'F', 1913, 'Mary', 21),
 ('AK', 'F', 1913, 'Elizabeth', 9),
 ('AK', 'F', 1913, 'Margaret', 8),
 ('AK', 'F', 1913, 'H

**Task 2 (for everyone):** Write a query that returns the statistics for the name William (5pt). Use the [the timeit package](https://docs.python.org/3/library/timeit.html) to measure the time it takes the query to run (5pt). **Bonus:** [Create an index](https://www.w3schools.com/sql/sql_create_index.asp)  on the _Name_ column and use the [the timeit package](https://docs.python.org/3/library/timeit.html) to measure the time it takes the query to run with the index (5pt)

In [59]:
import time
import timeit

def test():
    query = "SELECT COUNT(*) FROM Names WHERE Name='William'"
    print("There are {} williams".format(c.execute(query).fetchone()[0]))

c.execute("DROP INDEX IF EXISTS idx_name")
print("Time without index:{}".format(timeit.timeit("test()", globals=globals(), number=1)))

c.execute('''CREATE INDEX IF NOT EXISTS idx_name
             ON Names (name);''')
print("Time with index:{}".format(timeit.timeit("test()", globals=globals(), number=1)))

There are 6726 williams
Time without index:0.44438184000000547
There are 6726 williams
Time with index:0.0006709849999424478


### <span style="color:red"> Please answer only **one** of the following questions according to your ID number (use the formula **<YOUR_ID> mod 4 +1**) </span>

In [6]:
# which question to answer - put your ID number and run the code 
your_id  = "316460443"
q = int(your_id) % 4 + 1
print("You need to answer question number %s" % q)

You need to answer question number 4


***Question 1:*** Write a function that returns how many babies were born in a given state in a given year.
Use it to calculate the number of babies born in LA in 1950 (10pt)

***Question 2:*** Write a function that returns how many male babies were born between a given range of years.
Use it to calculate how many babies were born between 1970 and 1975  (10pt)

**Question 3:** Write a function that returns the most common female name in a given state. Use it to calculate the most common female name in Wasington in 1987 (10pt)

**Question 4:** Write a function that returns how many male babies named _William_ where born in a given state in a given year. Use it to find the state in which the highest number of babies _William_ where born in 1999 (10pt)

In [72]:
def find_babies(year, state=None, gender='M', name='William'):
    num = c.execute(
    f'''SELECT COUNT(*)
        FROM Names
        WHERE State='{state}' AND
              Gender='{gender}'AND
              Name='{name}' AND
              Year={year}
    '''
    ).fetchone()[0]
    print(f'Number of babies with name={name}, year={year}, gender={gender} is {num} in state={state}')

def find_babies_max(year, state=None, gender='M', name='William'):
    num, state = c.execute(
    f'''SELECT COUNT(*), State
        FROM Names
        WHERE Gender='{gender}'AND
              Name='{name}' AND
              Year={year}
        ORDER BY 1 DESC
    '''
    ).fetchone()
    print(f'Max number of babies with name={name}, year={year}, gender={gender} is {num} in state={state}')


find_babies(year='1999', state='AK')
# implemented more efficiently using single query
# could also be implemented with for loop
find_babies_max(year=1999)

Number of babies with name=William, year=1999, gender=M is 1 in state=AK
Max number of babies with name=William, year=1999, gender=M is 51 in state=WY


**Question (for everyone):** For the state of NY write code that calculates the second most popular female/male names in each decade (10pt). **Bonus**: Visualize it somehow using Matplotlib (5pt)

In [108]:
def find_popular_in_decade(decade, gender, state='NY'):
    print(c.execute(
    f'''SELECT SUBSTR(Year, 1, 3) || '0', Gender, Name, COUNT(*)
        FROM Names
        WHERE State='{state}' AND
              Gender='{gender}' AND
              (SUBSTR(Year, 1, 3) || '0')='{decade}'
        GROUP BY SUBSTR(Year, 1, 3), Name
        ORDER BY 4 DESC LIMIT 1
    '''
    ).fetchall()[0])

for decade in range(1910, 2010, 10):
    find_popular_in_decade(decade=decade, gender='M')
    find_popular_in_decade(decade=decade, gender='F')

('1910', 'M', 'Aaron', 10)
('1910', 'F', 'Ada', 10)
('1920', 'M', 'Aaron', 10)
('1920', 'F', 'Ada', 10)
('1930', 'M', 'Aaron', 10)
('1930', 'F', 'Ada', 10)
('1940', 'M', 'Aaron', 10)
('1940', 'F', 'Abby', 10)
('1950', 'M', 'Aaron', 10)
('1950', 'F', 'Abbe', 10)
('1960', 'M', 'Aaron', 10)
('1960', 'F', 'Abby', 10)
('1970', 'M', 'Aaron', 10)
('1970', 'F', 'Abby', 10)
('1980', 'M', 'Aaron', 10)
('1980', 'F', 'Abbey', 10)
('1990', 'M', 'Aaron', 10)
('1990', 'F', 'Abbey', 10)
('2000', 'M', 'Aaron', 10)
('2000', 'F', 'Aaliyah', 10)


# 2. Flavors of Cacao Dataset (15pt)

Using the [Flavors of Cacao](https://www.kaggle.com/rombikuboktaeder/choco-flavors) dataset, answer the following questions:

**Question 1:** Write a function that returns the number of bars manufactured where the bars' BroadBean Origin is a given country. Use the function to calculate the number of bars where BroadBean Origin is 'Fiji' (15pt)

**Question 2:** Write a function that returns the maximal and average cocoa percentage in a bar manufactured by a company in a specific country. Use the function to calculate the maximal and average cocoa percentage in bars manufactured by a Swiss company (15pt).

**Question 3:** Calculate the second most common bean type(s) and the most rare bean type(s) (15
pt)

**Question 4:** Calculate the number of reviews and the average rating in each year. Calculate the number of reviews and the average rating of each company in each year (15pt)

In [112]:
!mkdir /root/.kaggle/
import json
import os

# Installing the Kaggle package
!pip install kaggle 

#Important Note: complete this with your own key - after running this for the first time remmember to **remove** your API_KEY
api_token = {"username":"liaddekel","key":"f108a5e28c6e44704d469f7ae7614d16"}


# creating kaggle.json file with the personal API-Key details 
# You can also put this file on your Google Drive
with open('/root/.kaggle/kaggle.json', 'w') as file:
  json.dump(api_token, file)
!chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [143]:
# download and unzip dataset
!kaggle datasets list -s choco_flavors
!kaggle datasets download rombikuboktaeder/choco-flavors -p ./datasets/
!unzip ./datasets/choco-flavors.zip -d ./datasets/choco-flavors/

ref                             title          size  lastUpdated          downloadCount  
------------------------------  -------------  ----  -------------------  -------------  
rombikuboktaeder/choco-flavors  choco_flavors  30KB  2018-04-01 04:36:29            478  
choco-flavors.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  ./datasets/choco-flavors.zip
  inflating: ./datasets/choco-flavors/flavors_of_cacao.csv  


In [141]:
!pip install pony

     |████████████████████████████████| 286kB 2.8MB/s 
  Created wheel for pony: filename=pony-0.7.13-cp36-none-any.whl size=345363 sha256=9edd43a317ca1b77a74b67ba43293e941bdf11787c5eae6f9f602d8f4757ce3c
  Stored in directory: /root/.cache/pip/wheels/ca/c7/66/a9192d0ea4bcb17a25164284ec89b4563a6afbf7333947c0ea
Successfully built pony


In [2]:
from pony.orm import *
# Creating a new database
db = Database()
db.bind(provider='sqlite', filename='/content/datasets/choco-flavors/choco-flavors.pony.db', create_db=True)

class ChocoFlavor(db.Entity):
    Company = Optional(str)
    BeanOrigin = Required(str)
    REF = Required(int)
    ReviewDate = Required(int)
    CocoaPrecent = Required(str)
    Location = Required(str)
    Rating = Required(float)
    BeanType = Optional(str)
    BroadBeanOrigin = Optional(str)
     
set_sql_debug(True) # helps to see what SQL commands are running
db.generate_mapping(create_tables=True) # create tables

GET CONNECTION FROM THE LOCAL POOL
PRAGMA foreign_keys = false
BEGIN IMMEDIATE TRANSACTION
SELECT "ChocoFlavor"."id", "ChocoFlavor"."Company", "ChocoFlavor"."BeanOrigin", "ChocoFlavor"."REF", "ChocoFlavor"."ReviewDate", "ChocoFlavor"."CocoaPrecent", "ChocoFlavor"."Location", "ChocoFlavor"."Rating", "ChocoFlavor"."BeanType", "ChocoFlavor"."BroadBeanOrigin"
FROM "ChocoFlavor" "ChocoFlavor"
WHERE 0 = 1

COMMIT
PRAGMA foreign_keys = true
CLOSE CONNECTION


In [0]:
! rm -rf '/content/datasets/choco-flavors/choco-flavors.pony.db'

In [23]:
import pandas
import numpy as np

# pandas can find csv inside the zip itself
df = pandas.read_csv("./datasets/choco-flavors.zip")
df = df.replace(np.nan, '', regex=True)

for idx, row in df.iterrows():
    ChocoFlavor(
            Company=row['Company\xa0\n(Maker-if known)'],
            BeanOrigin=row['Specific Bean Origin\nor Bar Name'],
            REF=row['REF'],
            ReviewDate=row['Review\nDate'],
            CocoaPrecent=row['Cocoa\nPercent'],
            Location=row['Company\nLocation'],
            Rating=row['Rating'],
            BeanType=row['Bean\nType'],
            BroadBeanOrigin=row['Broad Bean\nOrigin'] if not type(row['Broad Bean\nOrigin']) is float else ""
    )
show(ChocoFlavor)
commit()

class ChocoFlavor(Entity):
    id = PrimaryKey(int, auto=True)
    Company = Optional(str, default='')
    BeanOrigin = Required(str)
    REF = Required(int)
    ReviewDate = Required(int)
    CocoaPrecent = Required(str)
    Location = Required(str)
    Rating = Required(float)
    BeanType = Optional(str, default='')
    BroadBeanOrigin = Optional(str, default='')


In [25]:
# number of reviews and average rating per year
"""
    SELECT "c"."ReviewDate", COUNT(DISTINCT "c"."id"), AVG("c"."Rating")
    FROM "ChocoFlavor" "c"
    GROUP BY "c"."ReviewDate
"""
print(list(select((c.ReviewDate, count(c), avg(c.Rating)) for c in ChocoFlavor)))

# number of reviews and average rating per company per year
"""
    SELECT "c"."Company", COUNT(DISTINCT "c"."id"), AVG("c"."Rating")
    FROM "ChocoFlavor" "c"
    GROUP BY "c"."Company"
"""
list(select((c.Company, c.ReviewDate, count(c), avg(c.Rating)) for c in ChocoFlavor))

SELECT "c"."Company", "c"."ReviewDate", COUNT(DISTINCT "c"."id"), AVG("c"."Rating")
FROM "ChocoFlavor" "c"
GROUP BY "c"."Company", "c"."ReviewDate"



[('A. Morin', 2012, 20, 3.625),
 ('A. Morin', 2013, 110, 3.3181818181818183),
 ('A. Morin', 2014, 50, 3.5),
 ('A. Morin', 2015, 40, 3.1875),
 ('A. Morin', 2016, 10, 3.75),
 ('AMMA', 2010, 40, 3.5625),
 ('AMMA', 2013, 10, 3.25),
 ('Acalli', 2015, 20, 3.75),
 ('Adi', 2011, 40, 3.25),
 ('Aequare (Gianduja)', 2009, 20, 2.875),
 ('Ah Cacao', 2009, 10, 3.0),
 ("Akesson's (Pralus)", 2010, 20, 2.75),
 ("Akesson's (Pralus)", 2011, 10, 3.75),
 ('Alain Ducasse', 2013, 20, 2.5),
 ('Alain Ducasse', 2014, 30, 2.8333333333333335),
 ('Alexandre', 2017, 40, 3.5),
 ('Altus aka Cao Artisan', 2013, 50, 3.0),
 ('Altus aka Cao Artisan', 2016, 50, 2.7),
 ('Amano', 2007, 30, 3.4166666666666665),
 ('Amano', 2008, 10, 2.75),
 ('Amano', 2009, 10, 3.0),
 ('Amano', 2010, 30, 3.5833333333333335),
 ('Amano', 2011, 10, 4.0),
 ('Amatller (Simon Coll)', 2009, 40, 2.875),
 ('Amazona', 2013, 20, 3.375),
 ('Ambrosia', 2015, 60, 3.25),
 ('Amedei', 2006, 20, 4.5),
 ('Amedei', 2007, 100, 3.725),
 ('Amedei', 2012, 10, 3.75),



# 3. Kickstarter Projects Dataset (25pt)

Using the [Kickstarter Projects Dataset](https://www.kaggle.com/kemical/kickstarter-projects#ks-projects-201801.csv), answer the following questions:

**Task 1 (for everyone):** Load the dataset to SQLite DB using [PonyORM](https://ponyorm.org) (10pt)

### <span style="color:red"> Please answer only **one** of the following questions according to your ID number (use the formula **<YOUR_ID> mod 3 +1**) </span>

In [0]:
# which question to answer - put your ID number and run the code 
your_id  = "<fill_your_id>"
q = int(your_id) % 3 + 1
print("You need to answer question number %s" % q)

**Question 1:** On average which project category received the highest number of backers? (15 pt)

**Question 2:** On average which project category received the highest pledged USD? (15 pt)

**Question 3:** In which month occurred the highest number of projects? (15 pt)

## 4. Oscars Datasets (10pt)

Using the [Oscars Dataset](https://www.kaggle.com/theacademy/academy-awards), please answer only one of the following questions (you can chose):

**Question 1:** Who is the female actress with the most Oscar nominees? (10pt)

**Question 2:** Who is the male director with the most Oscar nominees? (10pt)

**Question 3:** Which top-10 movies received the highest number of Oscar nominees? (10pt)

**Question 4:** Write a function that receives an actor's name and returns the actor’s number of Oscar nominees. Use the function to calculate the number of times Leonardo DiCaprio was a nominee (10pt)

## 5. Select a Dataset (15pt)

**Open Question:** Select an interesting dataset and use SQL to discover something interesting (15pt). **Bonus:** Use BigQuery (2pt)